In [ ]:
from google.colab import drive
import os
import shutil
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
import warnings

drive.mount('./gdrive', force_remount=True)
warnings.filterwarnings(action='ignore')

Mounted at ./gdrive


In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/투빅스/9주차/tobigs_week9_plant_leaf_data.zip (Unzipped Files)/')

original_dataset_dir = './tobigs_week9_plant_leaf'
classes_list = os.listdir(original_dataset_dir)

In [ ]:
base_dir = './splitted'
#os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
#os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'val')
#os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
#os.mkdir(test_dir)

In [ ]:
#for cls in classes_list:
#  os.mkdir(os.path.join(train_dir, cls))
#  os.mkdir(os.path.join(validation_dir, cls))
#  os.mkdir(os.path.join(test_dir, cls))

In [ ]:
"""
for cls in classes_list:
  path = os.path.join(original_dataset_dir, cls)
  fnames = os.listdir(path)

  train_size = math.floor(len(fnames) * 0.7)
  validation_size = math.floor(len(fnames) * 0.2)
  test_size = math.floor(len(fnames) * 0.1)

  train_fnames = fnames[:train_size]
  print("Train size(", cls, "): ", len(train_fnames))
  for fname in train_fnames:
    src = os.path.join(path, fname)
    dst = os.path.join(os.path.join(train_dir, cls), fname)
    shutil.copyfile(src, dst)
  
  validation_fnames = fnames[train_size:(validation_size + train_size)]
  print("Validation size(",cls,"): ", len(validation_fnames))
  for fname in validation_fnames:
    src = os.path.join(path, fname)
    dst = os.path.join(os.path.join(validation_dir, cls), fname)
    shutil.copyfile(src, dst)
        
  test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]
  print("Test size(",cls,"): ", len(test_fnames))
  for fname in test_fnames:
      src = os.path.join(path, fname)
      dst = os.path.join(os.path.join(test_dir, cls), fname)
      shutil.copyfile(src, dst)
"""

Train size( Cherry___healthy ):  597
Validation size( Cherry___healthy ):  170
Test size( Cherry___healthy ):  85
Train size( Pepper,_bell___Bacterial_spot ):  697
Validation size( Pepper,_bell___Bacterial_spot ):  199
Test size( Pepper,_bell___Bacterial_spot ):  99
Train size( Apple___healthy ):  1151
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  164
Train size( Potato___healthy ):  106
Validation size( Potato___healthy ):  30
Test size( Potato___healthy ):  15
Train size( Cherry___Powdery_mildew ):  736
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew ):  105
Train size( Strawberry___Leaf_scorch ):  776
Validation size( Strawberry___Leaf_scorch ):  221
Test size( Strawberry___Leaf_scorch ):  110
Train size( Pepper,_bell___healthy ):  1034
Validation size( Pepper,_bell___healthy ):  295
Test size( Pepper,_bell___healthy ):  147
Train size( Strawberry___healthy ):  319
Validation size( Strawberry___healthy ):  91
Test size( S

# 베이스라인 모델

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256 
EPOCH = 30 

In [ ]:
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()]) 
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base) 
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
# VGG16 구조에서 classifier부분의 구조를 GlobalAveragePooling + Linear로 조금 변경하였습니다.
class Net(nn.Module): 
  def __init__(self): 
    super(Net, self).__init__() 

    self.relu = nn.ReLU()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.conv4 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.conv5 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.conv6 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.conv7 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.conv8 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.conv9 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.conv10 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.pool4 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.conv11 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.conv12 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.conv13 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1, bias = True)
    self.pool5 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    
    self.gap = nn.AdaptiveAvgPool2d(output_size = (1, 1))

    self.fc1 = nn.Linear(in_features = 512, out_features = 256, bias = True)
    self.dropout1 = nn.Dropout(0.2)
    self.fc2 = nn.Linear(in_features = 256, out_features = 128, bias = True)
    self.dropout2 = nn.Dropout(0.2)
    self.fc3 = nn.Linear(in_features = 128, out_features = 21, bias = False)

  def forward(self, x):  
    x = self.relu(self.conv1(x))
    x = self.pool1(self.relu(self.conv2(x)))
    
    x = self.relu(self.conv3(x))
    x = self.pool2(self.relu(self.conv4(x)))

    x = self.relu(self.conv5(x))
    x = self.relu(self.conv6(x))
    x = self.pool3(self.relu(self.conv7(x)))

    x = self.relu(self.conv8(x))
    x = self.relu(self.conv9(x))
    x = self.pool4(self.relu(self.conv10(x)))

    x = self.relu(self.conv11(x))
    x = self.relu(self.conv12(x))
    x = self.pool5(self.relu(self.conv13(x)))
    
    x = self.gap(x)
    x = x.view(-1, 512)
    x = self.dropout1(self.relu(self.fc1(x)))
    x = self.dropout2(self.relu(self.fc2(x)))
    x = self.fc3(x)
    return x

model_base = Net().to(DEVICE)  
optimizer = optim.Adam(model_base.parameters(), lr=0.001) 
criterion = nn.CrossEntropyLoss().to(DEVICE)

In [ ]:
def train(model, train_loader, optimizer):
  model.train()

  training_loss = 0
  training_acc = 0

  total_batch = len(train_loader)

  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE) 
    optimizer.zero_grad()
    output = model(data)  
    pred = output.data.max(dim = 1, keepdim = True)[1]

    loss = criterion(output, target)  #Cross Entropy Loss 사용했습니다
    loss.backward()
    optimizer.step()    

    training_loss += loss / total_batch
    training_acc += pred.eq(target.data.view_as(pred)).cpu().sum() / BATCH_SIZE / total_batch
  
  return training_loss, training_acc

In [ ]:
def evaluate(model, test_loader):
  model.eval()

  test_loss = 0
  test_acc = 0  

  total_batch = len(test_loader)

  with torch.no_grad(): 
    for data, target in test_loader:  
      data, target = data.to(DEVICE), target.to(DEVICE)  
      output = model(data) 
      pred = output.max(1, keepdim=True)[1]
      
      loss = criterion(output, target)
      
      test_loss += loss / total_batch
      test_acc += pred.eq(target.data.view_as(pred)).cpu().sum() / BATCH_SIZE / total_batch 
   
  return test_loss, test_acc

In [ ]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer,num_epochs = 30):
  best_acc = 0.0  
  best_model_wts = copy.deepcopy(model.state_dict()) 
 
  for epoch in range(1, num_epochs + 1):
    since = time.time()  
    train_loss, train_acc = train(model, train_loader, optimizer)
    val_loss, val_acc = evaluate(model, val_loader)
        
    if val_acc > best_acc: 
      best_acc = val_acc 
      best_model_wts = copy.deepcopy(model.state_dict())
        
    time_elapsed = time.time() - since 
    print('-------------- epoch {} ----------------'.format(epoch))
    print('train Loss: {:.4f}, Accuracy: {:.2f}'.format(train_loss, train_acc))   
    print('val Loss: {:.4f}, Accuracy: {:.2f}'.format(val_loss, val_acc))
    print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 

  model.load_state_dict(best_model_wts)  
  return model

In [ ]:
base = train_baseline(model_base, train_loader, val_loader, optimizer, 30)  	
torch.save(base,'baseline.pt')

-------------- epoch 1 ----------------
train Loss: 2.9701, Accuracy: 0.08
val Loss: 2.8617, Accuracy: 0.06
Completed in 0m 33s
-------------- epoch 2 ----------------
train Loss: 2.7895, Accuracy: 0.11
val Loss: 2.8231, Accuracy: 0.14
Completed in 0m 33s
-------------- epoch 3 ----------------
train Loss: 2.7261, Accuracy: 0.13
val Loss: 2.7282, Accuracy: 0.14
Completed in 0m 33s
-------------- epoch 4 ----------------
train Loss: 2.6764, Accuracy: 0.14
val Loss: 2.7347, Accuracy: 0.16
Completed in 0m 33s
-------------- epoch 5 ----------------
train Loss: 2.5249, Accuracy: 0.17
val Loss: 2.3524, Accuracy: 0.21
Completed in 0m 32s
-------------- epoch 6 ----------------
train Loss: 2.2890, Accuracy: 0.22
val Loss: 2.3907, Accuracy: 0.28
Completed in 0m 33s
-------------- epoch 7 ----------------
train Loss: 2.0615, Accuracy: 0.28
val Loss: 1.9720, Accuracy: 0.33
Completed in 0m 33s
-------------- epoch 8 ----------------
train Loss: 1.9132, Accuracy: 0.33
val Loss: 2.4327, Accuracy: 0

# Transfer Learning 모델 학습

In [ ]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize((64,64)), 
                                 transforms.RandomResizedCrop(size = (64, 64), scale = (0.8, 1)),
                                 transforms.RandomAffine(degrees = 30),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    
    'val': transforms.Compose([transforms.Resize((64,64)),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
}

In [ ]:
train_dataset = ImageFolder(root='./splitted/train', transform=data_transforms['train']) 
val_dataset = ImageFolder(root='./splitted/val', transform=data_transforms['val'])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
# model_base와 비교하기 위하여 동일하게 VGG16의 특징 추출기만 가져온 후 GAP + Linear로 구조를 변경했습니다.
# 훈련집합의 수가 적지 않고 Imagenet과 task가 상이하고 판단하여 특징추출기의 가중치만 가져온 후 전체 모델을 학습시킵니다. 
class TransferNet(nn.Module): 
  def __init__(self): 
    super(TransferNet, self).__init__() 

    self.features = models.vgg16(pretrained = True).features

    self.gap = nn.AdaptiveAvgPool2d(output_size = (1, 1))

    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(in_features = 512, out_features = 256, bias = True)
    self.dropout1 = nn.Dropout(0.2)
    self.fc2 = nn.Linear(in_features = 256, out_features = 128, bias = True)
    self.dropout2 = nn.Dropout(0.2)
    self.fc3 = nn.Linear(in_features = 128, out_features = 21, bias = False)
  
  def forward(self, x):
    x = self.features(x)
    x = self.gap(x)
    x = x.view(-1, 512)
    x = self.dropout1(self.relu(self.fc1(x)))
    x = self.dropout2(self.relu(self.fc2(x)))
    x = self.fc3(x)
    return x

model_transfer = TransferNet().to(DEVICE)  
optimizer = optim.Adam(model_transfer.parameters(), lr=0.001) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', factor = 0.5, patience = 5)
criterion = nn.CrossEntropyLoss().to(DEVICE)

In [ ]:
# scheduler를 포함하는 함수로 train함수를 변경해줍니다
def train_baseline(model ,train_loader, val_loader, optimizer, scheduler, num_epochs = 30):
  best_acc = 0.0  
  best_model_wts = copy.deepcopy(model.state_dict()) 
 
  for epoch in range(1, num_epochs + 1):
    since = time.time()  
    train_loss, train_acc = train(model, train_loader, optimizer)
    val_loss, val_acc = evaluate(model, val_loader)
    scheduler.step(val_acc)

    if val_acc > best_acc: 
      best_acc = val_acc 
      best_model_wts = copy.deepcopy(model.state_dict())
        
    time_elapsed = time.time() - since 
    print('-------------- epoch {} ----------------'.format(epoch))
    print('train Loss: {:.4f}, Accuracy: {:.2f}'.format(train_loss, train_acc))   
    print('val Loss: {:.4f}, Accuracy: {:.2f}'.format(val_loss, val_acc))
    print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 

  model.load_state_dict(best_model_wts)  
  return model

In [ ]:
trans = train_baseline(model_transfer, train_loader, val_loader, optimizer, scheduler, 30)
torch.save(trans,'transfer.pt')

-------------- epoch 1 ----------------
train Loss: 2.8913, Accuracy: 0.09
val Loss: 2.4460, Accuracy: 0.12
Completed in 0m 38s
-------------- epoch 2 ----------------
train Loss: 2.2370, Accuracy: 0.22
val Loss: 1.8071, Accuracy: 0.36
Completed in 0m 38s
-------------- epoch 3 ----------------
train Loss: 1.6057, Accuracy: 0.41
val Loss: 1.3927, Accuracy: 0.50
Completed in 0m 38s
-------------- epoch 4 ----------------
train Loss: 1.2588, Accuracy: 0.54
val Loss: 1.0750, Accuracy: 0.61
Completed in 0m 38s
-------------- epoch 5 ----------------
train Loss: 0.9869, Accuracy: 0.62
val Loss: 0.8458, Accuracy: 0.65
Completed in 0m 38s
-------------- epoch 6 ----------------
train Loss: 0.7420, Accuracy: 0.72
val Loss: 0.5768, Accuracy: 0.78
Completed in 0m 38s
-------------- epoch 7 ----------------
train Loss: 0.5666, Accuracy: 0.79
val Loss: 0.5610, Accuracy: 0.81
Completed in 0m 37s
-------------- epoch 8 ----------------
train Loss: 0.4158, Accuracy: 0.85
val Loss: 0.3746, Accuracy: 0

# 모델 평가

In [ ]:
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()]) 
test_base = ImageFolder(root='./splitted/test',transform=transform_base)  
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
transform_trans = transforms.Compose([transforms.Resize((64,64)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_trans = ImageFolder(root='./splitted/test',transform=transform_trans)  
test_loader_trans = torch.utils.data.DataLoader(test_trans, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
baseline=torch.load('baseline.pt') 
baseline.eval()  
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('test acc:  ', test_accuracy)

test acc:   tensor(0.8608)


In [ ]:
vgg16=torch.load('transfer.pt') 
vgg16.eval()  
test_loss, test_accuracy = evaluate(vgg16, test_loader_trans)

print('test acc:  ', test_accuracy)

test acc:   tensor(0.9233)


* 학습속도 측면에서 전이학습의 학습속도(수렴속도)가 빨랐다. 동일한 epoch동안 학습 결과 basemodel의 validation_accuracy는 0.9, test_accuracy는 0.86, transfermodel의 validation_accuracy는 0.98, test_accuracy는 0.92였다. ==>
basemodel의 학습 epoch를 더 키운다면 결과적으로는 비슷한 성능을 보일것으로 보인다.
* 전이학습 시 cropping과 회전을 통해 데이터를 augmentation했으나 validation_accuracy와 train_accuracy에 비해 test_accuracy가 확연히 낮아 과적합으로 보인다 ==> 모델 내부에서 규제가 더 필요한것으로 보인다.